In [4]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00


In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [ ]:
from google.colab import files
uploaded = files.upload()

In [16]:
filename1 = '/content/merged_essays_final.csv'
df = pd.read_csv(filename1)

In [26]:
df = df[(df['Class'] != 0) | (df['Class'] != 1)]

In [22]:
df = df.dropna(subset=['Text'])

In [20]:
df=df.dropna()
df.shape

(5234, 2)

In [27]:
df.head()

,Class,Text
0,0,Money Making garden sculptures is a business i...
1,0,Table of contents Introduction Body Regulation...
2,0,Profit is the lifeblood of every form of busin...
3,0,Business ethics usually implemented through co...
4,0,Companies can manage ethics in the workplace b...


In [18]:
df=df[['Class','Text']]

In [28]:
text = df.Text.values
labels = df.Class.values

In [29]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [30]:
def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════════════╤═════════════╕
│ Tokens           │   Token IDs │
╞══════════════════╪═════════════╡
│ pharmacy         │       13882 │
├──────────────────┼─────────────┤
│ technicians      │       20202 │
├──────────────────┼─────────────┤
│ are              │        2024 │
├──────────────────┼─────────────┤
│ considered       │        2641 │
├──────────────────┼─────────────┤
│ by               │        2011 │
├──────────────────┼─────────────┤
│ many             │        2116 │
├──────────────────┼─────────────┤
│ as               │        2004 │
├──────────────────┼─────────────┤
│ an               │        2019 │
├──────────────────┼─────────────┤
│ important        │        2590 │
├──────────────────┼─────────────┤
│ part             │        2112 │
├──────────────────┼─────────────┤
│ of               │        1997 │
├──────────────────┼─────────────┤
│ any              │        2151 │
├──────────────────┼─────────────┤
│ successful       │        3144 │
├──────────────────┼

In [31]:
df = df[df['Class'].astype(str).str.isdigit()]

# Now, you can proceed with converting labels to integers
labels = df['Class'].astype(int)

In [34]:
df = df.reset_index(drop=True)
labels=df['Class']

In [33]:
df['Class'] = df['Class'].astype(int)
labels = torch.tensor(df['Class'])

In [35]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids'])
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [36]:
token_id[5230]

tensor([  101,  1996,  2412,  1011, 20607,  2088,  1997,  2974,  2038,  2904,
         1996,  2126,  1999,  2029,  2057,  2004,  3633, 11835,  2007,  2028,
         2178,  1998,  2256,  4044,  1012,  4816,  5733,  2031,  2468,  2062,
         6377,   102])

In [37]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒══════════════╤═════════════╤══════════════════╕
│ Tokens       │   Token IDs │   Attention Mask │
╞══════════════╪═════════════╪══════════════════╡
│ [CLS]        │         101 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ to           │        2000 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ many         │        2116 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ modern       │        2715 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ readers      │        8141 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ ,            │        1010 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ the          │        1996 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ science      │        2671 │                1 │
├──────────────┼─────────────┼──────────────────┤
│ -            │        1011 │                1 │


In [38]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx],
                          attention_masks[train_idx],
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx],
                        attention_masks[val_idx],
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [39]:

def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity


In [3]:
df.isna().sum()

NameError: ignored

In [40]:
!pip install torch torchvision torchaudio


In [41]:
import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, eps=1e-08)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(),
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [43]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids,
                             token_type_ids = None,
                             attention_mask = b_input_mask,
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids,
                              token_type_ids = None,
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:  50%|█████     | 1/2 [00:32<00:32, 32.41s/it]


	 - Train loss: 0.1774
	 - Validation Accuracy: 0.9675
	 - Validation Precision: 0.9484
	 - Validation Recall: 0.9934
	 - Validation Specificity: 0.9298



Epoch: 100%|██████████| 2/2 [01:05<00:00, 32.60s/it]


	 - Train loss: 0.0427
	 - Validation Accuracy: 0.9735
	 - Validation Precision: 0.9642
	 - Validation Recall: 0.9922
	 - Validation Specificity: 0.9461



In [48]:
# new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'
# new_sentence ='Over the last decade, cities around the world have been transitioning to greener public transportation options. While this is a major step in protecting our environment, it can be difficult for those living in urban areas who rely on cars and other motorized vehicles for their everyday lives. One of these options that has become increasingly popular is car-sharing services. Car-sharing services are programs offered by companies or municipalities which allow people to access a vehicle when they need one without having to own one themselves. This model of transportation provides many advantages over traditional methods such as reduced expenses, convenience, and environmental benefits. By accessing vehicles only when needed instead of owning them permanently, users can save money on both initial purchase costs and long-term maintenance fees like insurance and fuel charges. Additionally, car-sharing services provide flexibility with flexible rental periods from minutes up to days at a time so users can get where they need quickly without any hassle or commitment beyond what is necessary for their journey - perfect for short trips into town or long weekend getaways alike!. The sustainability aspect of car sharing should not be overlooked either; fewer cars on the road mean less dependence on fossil fuels which reduces emissions significantly helping us reach our goals of more sustainable lifestyles sooner rather than later. Moreover, given its low cost profile compared to buying an entire vehicle outright combined with its environmental friendliness makes it much easier even for those who may not normally consider green alternatives due to limited financial resources - furthering efforts towards greater widespread adoption across all communities regardless of socioeconomic status. In conclusion, while car ownership remains important in certain situations there are significant benefits available through utilizing car-sharing services - allowing us to remain mobile while conserving resources and reducing pollution levels worldwide at an affordable rate! Whether as part of regular daily routines or occasional special occasions we should all strive to do our part in creating a healthier planet by considering alternative modes like shared electric vehicles whenever feasible.'
# We need Token IDs and Attention Mask for inference on the new sentence
new_sentence ='Title: Narendra Modi: India\'s Charismatic Leader Introduction Narendra Damodardas Modi, born on September 17, 1950, in Vadnagar, Gujarat, is undoubtedly one of the most significant and influential political figures in contemporary Indian history. He rose from humble beginnings to become the 14th Prime Minister of India, serving since May 2014. Narendra Modi\'s journey to political prominence is marked by his charisma, leadership qualities, and a vision for transforming India. In this essay, we will explore his life, political career, and the impact he has had on the nation.Early Life and Rise to Politics Narendra Modi\'s early life was marked by simplicity and struggle. Raised in a modest family, he worked at his father\'s tea stall and helped the family financially. His formative years were marked by a quest for knowledge, and he eventually joined the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist organization, which played a pivotal role in shaping his political ideology.Modi\'s entry into mainstream politics began in the 1980s when he joined the Bharatiya Janata Party (BJP). He gradually climbed the political ladder, holding various positions within the party. His administrative skills and organizational acumen soon garnered attention, and he was appointed as the Chief Minister of Gujarat in 2001.Chief Minister of Gujarat Narendra Modi\'s tenure as Chief Minister of Gujarat was marked by both praise and controversy. Under his leadership, Gujarat experienced impressive economic growth and development. The state attracted significant investment, and infrastructure improved significantly during his tenure. This period earned Modi the reputation of a pro-business and development-focused leader.However, Modi\'s tenure was also marred by controversy, particularly in relation to the 2002 Gujarat riots, which resulted in the loss of thousands of lives, mostly Muslims. Critics accused him of not doing enough to prevent the violence, while his supporters argue that he was not directly responsible for the events. These events have continued to be a subject of debate and controversy in Indian politics.Prime Ministership In 2014, Narendra Modi led the BJP to a historic victory in the Indian general elections. He became the Prime Minister of India with a decisive mandate, promising \\"Sabka Saath, Sabka Vikas\" (Collective Efforts, Inclusive Growth). His tenure as Prime Minister has been marked by several significant policy initiatives and achievements.1. **Economic Reforms**: Modi\'s government introduced several economic reforms, including the rollout of the Goods and Services Tax (GST), the \"Make in India\" campaign, and initiatives to improve the ease of doing business in India. These efforts aimed to boost India\'s economic growth and make it a global manufacturing hub.2. **Digital India**: The \"Digital India\" initiative aimed at transforming India into a digitally empowered society and knowledge economy. It focused on expanding internet connectivity, promoting e-governance, and digitizing various sectors.3. **Swachh Bharat Abhiyan**: A nationwide cleanliness campaign, Swachh Bharat Abhiyan, was launched to promote sanitation and hygiene. This program aimed to make India open defecation-free and improve public health.4. **Foreign Policy**: Modi\'s foreign policy initiatives, including strengthening ties with major global powers, revitalizing the \"Act East\" policy, and engaging in diplomacy with neighboring countries, have played a significant role in shaping India\'s global image.5. **Infrastructure Development**: The government invested heavily in infrastructure development, including the construction of roads, highways, and railways, to improve connectivity and stimulate economic growth.6. **Jan Dhan Yojana**: The Pradhan Mantri Jan Dhan Yojana aimed to provide financial inclusion to the masses by offering bank accounts, insurance, and pensions to millions of Indians who were previously unbanked.7. **COVID-19 Response**: During the COVID-19 pandemic, Modi\'s government implemented measures to combat the virus\'s spread and launched vaccination campaigns to protect the population.Critics have raised concerns about issues such as the government\'s handling of certain economic challenges, religious tensions, and freedom of expression. However, it is important to note that opinions on Modi\'s leadership are deeply polarized, and he continues to enjoy strong support from many segments of the Indian population.ConclusionNarendra Modi\'s journey from a humble background to the Prime Minister\'s office is a testament to his charisma, leadership, and political acumen. His tenure has seen India undergo significant changes, both economically and diplomatically. However, it has also been marked by controversies and debates. Regardless of one\'s opinion, it is undeniable that Narendra Modi has left an indelible mark on Indian politics, and his legacy will continue to be a subject of discussion and analysis for years to come.'
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Ai generated' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Human written'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  Title: Narendra Modi: India's Charismatic Leader Introduction Narendra Damodardas Modi, born on September 17, 1950, in Vadnagar, Gujarat, is undoubtedly one of the most significant and influential political figures in contemporary Indian history. He rose from humble beginnings to become the 14th Prime Minister of India, serving since May 2014. Narendra Modi's journey to political prominence is marked by his charisma, leadership qualities, and a vision for transforming India. In this essay, we will explore his life, political career, and the impact he has had on the nation.Early Life and Rise to Politics Narendra Modi's early life was marked by simplicity and struggle. Raised in a modest family, he worked at his father's tea stall and helped the family financially. His formative years were marked by a quest for knowledge, and he eventually joined the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist organization, which played a pivotal role in shaping hi

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
